In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd
from torch.utils.datasets import Dataset, DataLoader
import os
from PIL import Image
import numpy as np
import PIL

In [3]:
import torchvision
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = torchvision.models.resnext50_32x4d(pretrained=False, progress=True)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.fc1 = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.model(x)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
model = Net().to("cuda")
torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
sy

In [6]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice


AttributeError: module 'syft' has no attribute 'TorchHook'

In [4]:
type(bob.id)

str

In [5]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 20
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
classes = {}
pt = 0
for i in pd.read_csv('../csv/train.csv')['label'].unique():
    classes[i] = pt
    pt+=1
def get_onehot(label):
    return classes[label]

In [7]:
class XDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        label = self.df.iloc[idx]['label']
        img_name = os.path.join(self.root_dir,label,
                                str(self.df.iloc[idx]['image']))
        image = Image.open(img_name)
        image = PIL.ImageOps.grayscale(image)
        onehot = np.array(get_onehot(label))
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        if self.transform:
            image = self.transform(image)
        return (image,onehot)

In [8]:
df_train = pd.read_csv('../csv/train.csv')
df_train = df_train.sample(frac=1)
traindataset = XDataset(df_train,root_dir='../x-ray/train/',
                                        transform=
                                 transforms.Compose([transforms.Resize((28,28)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5,), (0.5,))]))

federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    traindataset.federate((bob,)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

df_test = pd.read_csv('../csv/test.csv')
df_test = df_test.sample(frac=1)
testdataset = XDataset(df_test,root_dir='../x-ray/test/',
                                        transform=
                                transforms.Compose([transforms.Resize((28,28)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5,), (0.5,))]))

test_loader = torch.utils.data.DataLoader(
    testdataset,
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [10]:
import copy

In [11]:
model = Net()

In [15]:
model.location

<VirtualWorker id:bob #objects:322>

In [13]:
model = Net()
model2 = copy.deepcopy(model)
model2.send(bob)
print(model2.location)
print(model.location)
model2 = copy.deepcopy(model)
model2.send(alice)
print(model2.location)
print(model.location)

<VirtualWorker id:bob #objects:324>
None
<VirtualWorker id:alice #objects:322>
None


In [6]:
model = Net().to(device="cuda:0")
model1 = copy.deepcopy(model)
model1.send(alice)
model1.get()
model1 = copy.deepcopy(model)
model1.send(bob)
model1.get()

Net(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [25]:
model = Net()
model2 = model
model2.send(bob)
print(model2.location)
print(model.location)

<VirtualWorker id:bob #objects:26>
<VirtualWorker id:bob #objects:26>


In [9]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    model.send(bob) # <-- NEW: send the model to the right location
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    model.get() # <-- NEW: get the model back

In [10]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [11]:
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cnn.pt")

Train Epoch: 1 [0/5248 (0%)]	Loss: 0.777822
Train Epoch: 1 [640/5248 (12%)]	Loss: 1.531856
Train Epoch: 1 [1280/5248 (24%)]	Loss: 0.511036
Train Epoch: 1 [1920/5248 (37%)]	Loss: 0.409821
Train Epoch: 1 [2560/5248 (49%)]	Loss: 0.422414
Train Epoch: 1 [3200/5248 (61%)]	Loss: 0.508148
Train Epoch: 1 [3840/5248 (73%)]	Loss: 0.329157
Train Epoch: 1 [4480/5248 (85%)]	Loss: 0.260077
Train Epoch: 1 [5120/5248 (98%)]	Loss: 0.230612


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_set_

In [11]:
device

device(type='cuda')

In [13]:
torch.Tensor([1,2,3]).cuda()

RuntimeError: cuda runtime error (999) : unknown error at /pytorch/aten/src/THC/THCGeneral.cpp:50